In [1]:
# intalling folium

In [1]:
# configuration here
CENTER = [29.961264, 76.826217] # brahma sarovar

In [62]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [2]:
import folium
m = folium.Map(location=CENTER, zoom_start=15, tiles="CartoDB dark_matter")
folium.Marker(CENTER, popup='<h1>I am the Center</h1>').add_to(m)

In [3]:
# Taking Three Random Points
CIRCLES = {
    1: {
        "center": [29.961542, 76.823127],
        "radius": 10,
        "color": 'red',
        "trust_val": 70
    },
    2: {
        "center": [29.961542, 76.823127],
        "radius": 20,
        "color": 'crimson',
        "trust_val": 69
    },
    3: {
        "center": [29.961542, 76.823127],
        "radius": 10*3,
        "color": 'green',
        "trust_val": 69
    }
}
radius =  10*3+15
x, y = CIRCLES[3]['center'][0], CIRCLES[3]['center'][1]
points = [
    (x, y), (x + radius, y + radius)
]


In [4]:
for circle in CIRCLES.values():
    folium.CircleMarker(
        location=circle['center'],
        radius=circle['radius'],
        fill=True,
        fill_color=circle['color']
    ).add_to(m)

In [318]:
folium.RegularPolygonMarker(
    location=circle['center'],
    radius=radius,
    color="red"
).add_to(m)

In [323]:
folium.RegularPolygonMarker(
    location=[circle['center'][0] + 10, circle['center'][1]],
    radius=100,
    color="red"
).add_to(m)

In [324]:
m

In [5]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [371]:
lower_left = [circle['center'][0], circle['center'][1]]
upper_right = [circle['center'][0] + 10, circle['center'][1] + 10]
m = folium.Map(zoom_start = 5, location=[55, 0],  tiles="CartoDB dark_matter")
grid = get_geojson_grid(upper_right, lower_left , n=6)

for i, geo_json in enumerate(grid):
#     color = plt.cm.Greens(i / len(grid))
    color = plt.cm.Greens((20- abs(i-15)) / len(grid))
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    popup = folium.Popup("example popup {}".format(abs(i-15)))
    gj.add_child(popup)

    m.add_child(gj)

In [7]:
m

In [9]:
lower_left = [circle['center'][0], circle['center'][1]]
upper_right = [circle['center'][0] + 10, circle['center'][1] + 10]
grid = get_geojson_grid(upper_right, lower_left , n=6)

In [13]:
import json
json.dump(grid, open("saren.json", "w"))

In [6]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes